In [2]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

In [3]:
def generate_embeddings(date_list, ticker_name='t1'):
    return pd.DataFrame({
        'date': date_list,
        'f1': np.random.randn(len(date_list)),
        'f2': np.random.randn(len(date_list)),
        'ticker': [ticker_name] * len(date_list),
    })

def generate_date_list(start_date=datetime(2018, 1, 1), end_date=datetime(2018, 1, 15), max_duplicates=2):
    date_list = []
    current_date = start_date
    while current_date <= end_date:
        for _ in range(max_duplicates):
            if np.random.random() < 0.5:
                date_list.append(current_date.strftime("%d/%m/%Y"))
        current_date += timedelta(days=1)
    print(f"Length of the date list: {len(date_list)}")
    return date_list

def create_true_date_list(start_date=datetime(2018, 1, 1), end_date=datetime(2018, 1, 15)):
    true_date_list = []
    current_date = start_date
    while current_date <= end_date:
        true_date_list.append(current_date.strftime("%d/%m/%Y"))
        current_date += timedelta(days=1)
    print(f"Length of the true date list: {len(true_date_list)}")
    return true_date_list

In [4]:
window = 7
min_periods = 1
true_date_list = create_true_date_list()

Length of the true date list: 15


# Rolling mean embedding of one ticker

## First version: mean by day and then mean of the last 7 days

In [5]:
np.random.seed(0)
date_list = generate_date_list()
embeddings = generate_embeddings(date_list).drop('ticker', axis=1)
embeddings = embeddings.groupby('date').mean()
embeddings = embeddings.reindex(true_date_list)
display(embeddings)
embeddings.rolling(window, min_periods=min_periods).mean()

Length of the date list: 10


,f1,f2
date,,
01/01/2018,NaN,NaN
02/01/2018,NaN,NaN
03/01/2018,0.864436,-0.887786
04/01/2018,-0.742165,-1.980796
05/01/2018,2.269755,-0.347912
06/01/2018,NaN,NaN
07/01/2018,NaN,NaN
08/01/2018,-0.704304,0.693320
09/01/2018,-0.187184,1.202380


,f1,f2
date,,
01/01/2018,NaN,NaN
02/01/2018,NaN,NaN
03/01/2018,0.864436,-0.887786
04/01/2018,0.061136,-1.434291
05/01/2018,0.797342,-1.072165
06/01/2018,0.797342,-1.072165
07/01/2018,0.797342,-1.072165
08/01/2018,0.421931,-0.630794
09/01/2018,0.300108,-0.264159


## Second version: mean of the last 7 days

In [27]:
np.random.seed(0)
date_list = generate_date_list()
date_list_2 = generate_date_list()
date_list_3 = generate_date_list()
embeddings = generate_embeddings(date_list)
embeddings_2 = generate_embeddings(date_list_2, ticker_name='t2')
embeddings_3 = generate_embeddings(date_list_3, ticker_name='t3')
embeddings = embeddings_inter = pd.concat((embeddings, embeddings_2, embeddings_3))
# display(embeddings)
embeddings = embeddings.groupby(['ticker', 'date']).mean()
def rolling_mean_group(group: pd.DataFrame):
    return group.set_index('date').reindex(true_date_list).drop('ticker', axis=1).rolling(window=window, min_periods=min_periods).mean()
    # return group.set_index('date').reindex(true_date_list).reset_index().drop('ticker', axis=1).rolling(window=window, min_periods=min_periods).mean()
embeddings.reset_index(drop=False).groupby('ticker').apply(rolling_mean_group).swaplevel().sort_index()

Length of the date list: 10
Length of the date list: 17
Length of the date list: 18


f1        f2
date       ticker                    
01/01/2018 t1           NaN       NaN
           t2      0.208275  1.480515
           t3      0.707055  1.663812
02/01/2018 t1           NaN       NaN
           t2      0.592457  1.674037
           t3      0.689675  0.375495
03/01/2018 t1      1.139401  0.465662
           t2      0.513760  1.418039
           t3      0.399374  0.217181
04/01/2018 t1     -0.047713 -0.535291
           t2      0.513760  1.418039
           t3      0.434343  0.047490
05/01/2018 t1      0.102306  0.139224
           t2      0.513760  1.418039
           t3      0.212608  0.024344
06/01/2018 t1      0.102306  0.139224
           t2      0.474954  1.194635
           t3      0.182478  0.305843
07/01/2018 t1      0.102306  0.139224
           t2      0.737138  0.902107
           t3      0.182478  0.305843
08/01/2018 t1     -0.117722  0.488751
           t2      0.869353  0.757505
           t3     -0.041339 -0.095584
09/01/2018 t1     -0.209947  0.355016
           t2      0.691306  0.509329
           t3     -0.048969 -0.020522
10/01/2018 t1     -0.547284  0.327355
           t2      0.669138  0.340201
           t3     -0.003122  0.034649
11/01/2018 t1     -0.318104  0.948554
           t2      0.281214  0.456602
           t3     -0.308357 -0.074037
12/01/2018 t1     -0.556069  0.095552
           t2      0.395911  0.443240
           t3     -0.122736 -0.135633
13/01/2018 t1     -0.403010  0.335277
           t2      0.400363  0.289073
           t3     -0.100262 -0.504196
14/01/2018 t1     -0.555438  0.187586
           t2      0.033781  0.554804
           t3      0.004780 -0.156520
15/01/2018 t1     -0.219712  0.124608
           t2      0.112919  0.415246
           t3      0.111551 -0.058481

In [81]:
embeddings_inter.sample(frac=1).sort_values(['ticker', 'date']).drop(['ticker', 'date'], axis=1).reset_index(drop=True) - embeddings_inter.drop(['ticker', 'date'], axis=1).reset_index(drop=True)

,f1,f2
0,0.00000,0.000000
1,0.00000,0.000000
2,0.00000,0.000000
3,0.00000,0.000000
4,0.00000,0.000000
5,0.00000,0.000000
6,0.00000,0.000000
7,0.00000,0.000000
8,0.00000,0.000000
9,0.00000,0.000000


In [83]:
embeddings_inter.reset_index(drop=True)

,date,f1,f2,ticker
0,03/01/2018,1.764052,0.144044,t1
1,04/01/2018,0.400157,1.454274,t1
2,05/01/2018,0.978738,0.761038,t1
3,08/01/2018,2.240893,0.121675,t1
4,08/01/2018,1.867558,0.443863,t1
5,09/01/2018,-0.977278,0.333674,t1
6,12/01/2018,0.950088,1.494079,t1
7,13/01/2018,-0.151357,-0.205158,t1
8,14/01/2018,-0.103219,0.313068,t1
9,15/01/2018,0.410599,-0.854096,t1


In [56]:
embeddings_inter.sample(frac=1).sort_values(['ticker', 'date'])

,date,f1,f2,ticker
0,03/01/2018,1.764052,0.144044,t1
1,04/01/2018,0.400157,1.454274,t1
2,05/01/2018,0.978738,0.761038,t1
3,08/01/2018,2.240893,0.121675,t1
4,08/01/2018,1.867558,0.443863,t1
5,09/01/2018,-0.977278,0.333674,t1
6,12/01/2018,0.950088,1.494079,t1
7,13/01/2018,-0.151357,-0.205158,t1
8,14/01/2018,-0.103219,0.313068,t1
9,15/01/2018,0.410599,-0.854096,t1


In [12]:
pd.DataFrame({
    'a': [1, 2, 3, 6],
    'b': [3, 4, 6, 8],
}).sample(frac=1, random_state=42)

,a,b
1,2,4
3,6,8
0,1,3
2,3,6
